In [1]:
import numpy as np                            
import matplotlib.pyplot as plt               
import cartopy.crs as ccrs                    
import cartopy.feature as cfeature           
import herbie     

In [2]:
lcc_proj = ccrs.LambertConformal(central_longitude=-95, 
                             central_latitude=35,
                             standard_parallels=[35])

pc_proj = ccrs.PlateCarree()

def synoptic_view():
    
    date=input('Date and Time? yyyy-mm-dd hh:mm')
    
    hour=int(input('Forecast hour?'))
    
    print(date)
    
    data=herbie.Herbie(date,model='gfs',product='pgrb2.0p25',fxx=hour)

    u250=data.xarray(':UGRD:250') #u wind at 250mb
    
    v250=data.xarray(':VGRD:250') #v wind at 250mb
    
    u250lon=u250['longitude'].values
    
    v250lat=v250['latitude'].values
    
    u250wind=u250['u'].values

    v250wind=v250['v'].values
    
    mag250=np.sqrt(u250wind**2 + v250wind**2) #magnitude of 250mb winds
    
    val = u250['valid_time'].dt.strftime('%Y-%m-%d %H00').values
    
    ini = u250['time'].dt.strftime('%Y-%m-%d %H00').values

    h250=data.xarray(':HGT:250') #geopotential heights at 250mb
   
    h250lon=h250['longitude'].values
   
    h250lat=h250['latitude'].values
   
    h250_values=h250['gh'].values

    mslp=data.xarray(':PRMSL:mean sea level')
    
    mslp_lon = mslp['longitude'].values
    
    mslp_lat = mslp['latitude'].values
    
    mslp_values=mslp['prmsl'].values

    fig, ax = plt.subplots(figsize=(20, 17), subplot_kw={'projection': lcc_proj})

    ax.add_feature(cfeature.LAND.with_scale('50m'))
    
    ax.add_feature(cfeature.OCEAN.with_scale('50m'))
   
    ax.add_feature(cfeature.LAKES.with_scale('50m'))
    
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
   
    ax.add_feature(cfeature.STATES.with_scale('50m'))
   
    ax.add_feature(cfeature.BORDERS.with_scale('50m'))
    
    ax.set_extent((-120, -73, 23, 50))
    
    jet_range=[30,80]
    
    levels = np.linspace(jet_range[0], jet_range[1], 50)
    
    p250=ax.contourf(u250lon, v250lat, mag250,transform=pc_proj, levels=levels, cmap='jet',zorder=3,alpha=.5)
    
    ax.contour(h250lon, h250lat, h250_values, transform=pc_proj, levels=30, colors='black',zorder=2)
    
    cbar250 = plt.colorbar(p250, ax=ax, orientation='horizontal',pad=.07)
    
    cbar250.set_label("250 mb wind speeds (kt)",fontsize=20)
   
    ax.contour(mslp_lon, mslp_lat, mslp_values, transform=pc_proj, levels=30, colors='red',zorder=1)

    plt.title(f"GFS Surface Pressure, 250mb Geopotential Heights and Wind Speed\nInitialized: {ini}\nValid: {val}", fontsize=20)
    
    plt.tight_layout()
    
    plt.show()
